In [1]:
from prepare import Simulation_Preparer

In [ ]:
prepper = Simulation_Preparer('3mxf.pdb', 'resname JQ1') # A Crystal Structure and A residue defining the ligand

In [ ]:
prepper.generate_topologies(save_as_jsons=True)

In [ ]:
prepper.generate_interchanges(["openff-2.1.0.offxml", "opc-1.0.1.offxml", "ff14sb_off_impropers_0.0.3.offxml"]) # Takes a while

In [ ]:
prepper.openmm_writeout() # Takes a while

In [ ]:
# ALL PREVIOUS MEMORY CAN BE CLEARED AT THIS POINT?

In [ ]:
complex_fns = ('complex_final.pdb', 'complex_final.xml')
solvent_fns = ('solvent_final.pdb', 'solvent_final.xml')
ligand_resname = 'resname UNK'
yank_output_dir = 'UNK/yankrun2'
yaml_file_fn = 'UNK/yank_script.yaml'
restraint_string = determine_restrained_residues(complex_fns[0], 4, ligand_resname)

with open(yaml_file_fn, 'w') as f:
    f.write(write_the_yaml(complex_fns, solvent_fns, ligand_resname, yank_output_dir, restraint_string))

In [ ]:
with open(yaml_file_fn, 'r') as f:
    my_yaml = yaml.load(f, Loader=YankLoader)
yaml_builder = yank.experiment.ExperimentBuilder(script=my_yaml)

In [ ]:
yaml_builder.run_experiments()

In [ ]:
import yank.analyze

In [ ]:
Analyzer = yank.analyze.ExperimentAnalyzer('UNK/yankrun2/experiments/')
results = Analyzer.auto_analyze()

In [ ]:
print(results)